### Librerias

In [ ]:
#%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: 'pandas==': Expected end or semicolon (after name and no valid version specifier)
    pandas==
          ^ (from line 2 of requirements.txt)


In [1]:
import os
import json
import pandas as pd
from docx import Document

from src.tabla_niveldep import construir_tabla1
from src.tabla_top import construir_tabla_top_dependencias
from src.utils import dataframe_a_json_tabla
from src.prompt_payloads import construir_payload_resumen, construir_payload_top_dependencias
from config.openai_config import openai_response
from prompts.loader import get_prompt, render_prompt
from src.docx_utils import insert_text_at_bookmark, insert_dataframe_at_bookmark

# Solo generar el CSV demo (uso interno, no obligatorio)
from tables.consultas_sql import exportar_improd_a_csv

In [2]:
## Conexión a la base de datos unicamente en entorno productivo
#test_connection()

In [3]:
## Usar solo en entorno interno para generar el CSV demo
#exportar_improd_a_csv("data/sample_improd.csv")

In [4]:
# Ejecutar consulta y obtener DataFrame
USE_DB = os.getenv("USE_DB", "0") == "1"

if USE_DB:
    from tables.consultas_sql import cargar_improd
    df = cargar_improd()
else:
    df = pd.read_csv("data/sample_improd.csv")

print("✅ Fuente:", "BD" if USE_DB else "CSV demo", "| filas:", len(df))

✅ Fuente: CSV demo | filas: 57737


In [5]:
USE_OPENAI = os.getenv("USE_OPENAI", "0") == "1"

In [7]:
def validar_openai_env():
    faltan = [k for k in ["AZURE_OPENAI_ENDPOINT","AZURE_OPENAI_API_KEY","AZURE_OPENAI_MODEL"] if not os.getenv(k)]
    if faltan:
        raise RuntimeError(
            "Faltan variables para Azure OpenAI: " + ", ".join(faltan) +
            ". Revisa el README y configura el .env."
        )

validar_openai_env()

### Tabla 1 - Párrafo 1

In [8]:
tabla_total_activos = construir_tabla1(df, filtro_improd="disciplinarios") 

json_1 = dataframe_a_json_tabla(tabla_total_activos, nombre_tabla="Disciplinarios TOTAL")

payload_1 = construir_payload_resumen(json_1)

# 1) Cargar plantillas
system_t = get_prompt("prompt_total", "system_v1.md")
user_t   = get_prompt("prompt_total", "resumen_v1.md")

# 2) Renderizar con tu payload
usr_msg = render_prompt(user_t, payload=payload_1, corte_hasta=payload_1["corte_hasta"])

# 3) Llamar al modelo (lee endpoint/key/deployment del .env)
texto_total_activos = openai_response(
    Role_system=system_t,
    Prompt=usr_msg,
    max_tokens=700,
    temperature=0.2
)
print(texto_total_activos)

En total, se registran 41.246 procesos disciplinarios activos, de los cuales 3.480 (8,44%) tienen más de cuatro años de antigüedad, según la columna de corte "HASTA 2021". Este grupo de procesos representa un porcentaje significativo dentro del total de casos activos, lo que puede implicar un riesgo en términos de prescripción, dado que los procesos disciplinarios tienen un límite temporal para ser resueltos.

La distribución de los casos entre los niveles Territorial y Central muestra una clara predominancia del nivel Territorial, que concentra 34.214 procesos, lo que representa una parte considerable del total. Dentro de este nivel, la Procuraduría Provincial es la dependencia con mayor carga, con 19.799 casos, lo que equivale al 57,87% del nivel Territorial. Le sigue la Procuraduría Regional con 10.995 casos (32,14%) y la Procuraduría Distrital con 3.407 casos (9,96%). En el nivel Central, que agrupa 7.032 procesos, la Procuraduría Delegada es la dependencia con más casos, sumando 5

### Tabla 2 - Párrafo 2

In [9]:
tabla_quejas = construir_tabla1(df, filtro_improd="quejas") 

json_quejas = dataframe_a_json_tabla(tabla_quejas, nombre_tabla="Quejas TOTAL")

payload_quejas = construir_payload_resumen(json_quejas)

# 1) Cargar plantillas
system_quejas = get_prompt("prompt_total", "system_v1.md")
user_quejas   = get_prompt("prompt_quejas", "resumen_v2.md")

# 2) Renderizar con tu payload
usr_msg_quejas = render_prompt(user_quejas, payload=payload_quejas, corte_hasta=payload_quejas["corte_hasta"])

# 3) Llamar al modelo (lee endpoint/key/deployment del .env)
texto_quejas = openai_response(
    Role_system=system_quejas,
    Prompt=usr_msg_quejas,
    max_tokens=700,
    temperature=0.2
)

print(texto_quejas)

Hasta el año 2021, el total de quejas con connotación disciplinaria activas asciende a 11.189 casos. De estos, el 2,52% corresponde a quejas que tienen más de cuatro años de antigüedad, sumando un total de 282 casos. En el año 2025 se registró el mayor número de quejas, alcanzando un total de 8.490. La distribución de las quejas se divide en dos niveles: TERRITORIAL y CENTRAL. En el nivel TERRITORIAL, se concentran 9.992 quejas, de las cuales el 2,27% son mayores a cuatro años. Las dependencias con mayor número de quejas en este nivel son la Procuraduría Provincial con 4.322 casos (43,25%), seguida por la Procuraduría Regional con 3.005 casos (30,07%) y la Procuraduría Distrital con 2.664 casos (26,66%). En el nivel CENTRAL, se registran 1.197 quejas, con un 4,59% de ellas siendo mayores a cuatro años. Las dependencias más destacadas en este nivel son la Procuraduría Delegada con 739 casos (61,74%), la Veeduría con 238 casos (19,88%) y la Sala Disciplinaria de Instrucción con 135 casos

### Tabla 3 - Párrafo 3

In [10]:
tabla_top_quejas = construir_tabla_top_dependencias(df, filtro_improd="quejas")

json_top_quejas = dataframe_a_json_tabla(tabla_top_quejas, nombre_tabla="Top 10 dependencias (quejas)")

payload_top_quejas = construir_payload_top_dependencias(json_top_quejas)

# 1) Cargar plantillas
system_top_quejas = get_prompt("prompt_total", "system_v1.md")
user_top_quejas   = get_prompt("prompt_quejas", "resumen_top_v1.md")

# 2) Renderizar con tu payload
usr_msg_top_quejas  = render_prompt(
    user_top_quejas,
    payload=payload_top_quejas,
    corte_hasta=payload_top_quejas["corte_hasta"],
    top1_dep=payload_top_quejas["top1"]["dependencia"],
    top1_pct=f'{payload_top_quejas["top1"]["pct"]}%',
    top3_pct=f'{payload_top_quejas["top3_share_pct"]}%'
)

# 3) Llamar al modelo (lee endpoint/key/deployment del .env)
texto_top_quejas = openai_response(
    Role_system=system_top_quejas,
    Prompt=usr_msg_top_quejas,
    max_tokens=700,
    temperature=0.2
)

print(texto_top_quejas)

Hasta el año 2021, el total de quejas con connotación disciplinaria en las diez principales dependencias asciende a 5404. La dependencia que lidera este grupo es la Procuraduría Primera Distrital de Instrucción, la cual acumula un total de 1438 quejas, representando aproximadamente el 26.61% del total. Las tres primeras dependencias en el ranking, que incluyen además a la Procuraduría Segunda Distrital de Instrucción y la Procuraduría Provincial de Instrucción Puerto Berrío, concentran aproximadamente el 57.57% de las quejas. En cuanto a la distribución anual de las quejas, se observa un incremento significativo en el año 2025, donde se registran 3443 quejas, siendo este el año con mayor volumen. Dentro de las dependencias, el año pico varía, destacando que la Procuraduría Segunda Distrital de Instrucción alcanza su máximo en 2025 con 1005 quejas, mientras que la Procuraduría Provincial de Instrucción Puerto Berrío tiene su mayor registro en 2024 con 114 quejas.


Lectura del documento plantilla

In [11]:
doc = Document("plantillas/Plantilla_Prueba.docx")

Insertar los textos con marcadores

In [12]:
# Texto (Arial 11)
insert_text_at_bookmark(doc, "<<texto_total_activos>>", texto_total_activos)
insert_text_at_bookmark(doc, "<<texto_quejas>>",  texto_quejas)
insert_text_at_bookmark(doc, "<<texto_top_quejas>>",     texto_top_quejas)

True

Insertar las tablas con marcadores

In [13]:
# Tabla (por ejemplo Top 10)
insert_dataframe_at_bookmark(doc, tabla_total_activos, "<<tabla_total_activos>>")
insert_dataframe_at_bookmark(doc, tabla_quejas, "<<tabla_quejas>>")
insert_dataframe_at_bookmark(doc, tabla_top_quejas, "<<tabla_top_quejas>>")

True

Guardar el documento final en la carpeta de salidas

In [15]:
doc.save("salidas/Informe_prueba.docx")